In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import sys 

sys.path.append('..')
sys.path.append('../src')

In [3]:
from data import AlfaDataModule
# from data import batches_generator
from models.task_model import TaskModule
from tasks.task_auto import AutoTask

In [4]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [5]:
from random import shuffle


dm = AlfaDataModule('/home/jovyan/romashka/data', shuffle=True)
task = AutoTask.get('default', task_type='non-text', tokenizer=tokenizer)

In [6]:
from transformers import get_linear_schedule_with_warmup

model = TaskModule(torch.optim.Adam, get_linear_schedule_with_warmup, head_type='next', task_name='next_transaction')

USING whisper


In [7]:
batch = next(iter(dm.val_dataloader()))

ValueError: persistent_workers option needs num_workers > 0

In [ ]:
# model.training_step(batch, 0)

In [ ]:
# model.task.generate_target(batch)

In [ ]:
# model.training_step(batch, 0)

### Model Next MCC

In [12]:
import random

In [13]:
target_batch = ['1', '2', '3', '4', '5', '6', '7', '8']

In [14]:
num_options = 2
answer_options = [str(i) for i in range(8)]

In [16]:
multichoice_separator = " - %s;"
question_end =  "Will the next transactions have merchant category code 2? Yes or No?"

In [17]:
target_batch_options = []  # a list of str target options
for gt_target in target_batch:
    target_options = {gt_target}
    while len(target_options) < num_options:
        target_options.add(random.sample(answer_options, k=1)[0])

    # shuffle
    target_options = random.sample(list(target_options), k=len(target_options))
    # connect with separator
    target_options = "".join([multichoice_separator % target for target in target_options])
    target_batch_options.append(target_options)

question_target_batch = [question_end + " OPTIONS:" + target_options for target_options in
                        target_batch_options] 

In [21]:
question_target_batch = [question_end + " OPTIONS:" + target_options for target_options in
                        target_batch_options] 

In [22]:
question_target_batch

['Will the next transactions have merchant category code 2? Yes or No? OPTIONS: - 1; - 0;',
 'Will the next transactions have merchant category code 2? Yes or No? OPTIONS: - 2; - 4;',
 'Will the next transactions have merchant category code 2? Yes or No? OPTIONS: - 3; - 5;',
 'Will the next transactions have merchant category code 2? Yes or No? OPTIONS: - 3; - 4;',
 'Will the next transactions have merchant category code 2? Yes or No? OPTIONS: - 0; - 5;',
 'Will the next transactions have merchant category code 2? Yes or No? OPTIONS: - 6; - 0;',
 'Will the next transactions have merchant category code 2? Yes or No? OPTIONS: - 7; - 6;',
 'Will the next transactions have merchant category code 2? Yes or No? OPTIONS: - 8; - 5;']

In [28]:
from src.transactions_qa.evaluation.eval_processings_utils import convert_to_numeric
from torchmetrics import Accuracy


answer = ['1', '2', '3', '4', 'suka']

In [30]:
metric = Accuracy(task='multiclass', num_classes=10, ignore_index=-1)

In [38]:
targets = torch.tensor((1, 1, 2))
preds = torch.tensor([-10, 1, 1])

In [39]:
metric(preds, targets)

tensor(0.3333)

In [41]:
torch.tensor(list(map(lambda x: convert_to_numeric(x, -1), answer)))

Value is not a number: suka


tensor([ 1,  2,  3,  4, -1])

### Debugging transformer

In [8]:
import os
from datasets.io.generator import GeneratorDatasetInputStream

In [9]:
data_dir = '/home/jovyan/romashka/data'

In [10]:
dir_with_datasets = os.listdir(os.path.join(data_dir, 'val_buckets'))
list_paths = sorted([os.path.join(data_dir, 'val_buckets', x)
                        for x in dir_with_datasets])

In [11]:
from src.data.alfa.components.data_generator import batches_generator

In [12]:
from datasets import IterableDataset as HFIterableDataset

In [13]:
dataset = HFIterableDataset.from_generator(batches_generator,
                                            gen_kwargs={
                                                'list_of_paths': list_paths,
                                                'min_seq_len': 0,
                                                'max_seq_len': 250,
                                                # 'shuffle': False,
                                            })

In [14]:
next(iter(dataset.with_format('torch')))

reading /home/jovyan/romashka/data/val_buckets/processed_chunk_000.pkl
/home/jovyan/romashka/data/val_buckets/processed_chunk_000.pkl = [0 1 2 3 4 5 6 7 8 9]


{'num_features': tensor([[[ 0.5222,  0.5242,  0.5242,  0.5396,  0.5247,  0.5043,  0.5271,
            0.5222,  0.5247,  0.5232,  0.2976,  0.4579,  0.4779]],
 
         [[ 0.9233,  0.8274,  0.7507,  0.6740,  0.5808,  0.4959,  0.4274,
            0.3260,  0.2521,  0.1644,  0.1644,  0.0767,  0.0767]],
 
         [[-0.0105,  8.7895,  7.0737,  7.1053,  8.5263,  7.8632,  6.3474,
            9.3368,  6.7789,  8.1474,  0.0000,  8.0632,  0.0000]]]),
 'cat_features': tensor([[[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]],
 
         [[ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4]],
 
         [[84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84]],
 
         [[14, 14,  4,  4,  4,  4,  4,  4,  4,  4,  4, 14, 14]],
 
         [[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2]],
 
         [[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]],
 
         [[ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3]],
 
         [[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2]],
 
   

In [16]:
gen = GeneratorDatasetInputStream(batches_generator, gen_kwargs={
                                                'list_of_paths': list_paths,
                                                'min_seq_len': 0,
                                                'max_seq_len': 225,
                                                'shuffle': False,
                                            }, streaming=True)

In [17]:
d = gen.read()

In [18]:
from datasets.download.streaming_download_manager import StreamingDownloadManager

In [19]:
data_dir = '/home/jovyan/romashka/data/train_buckets'

In [26]:
from datasets.download.download_config import DownloadConfig

In [27]:
dl_manager = StreamingDownloadManager(
            base_path=gen.builder.base_path,
            download_config=DownloadConfig(use_auth_token=gen.builder.use_auth_token, storage_options=gen.builder.storage_options),
            dataset_name=gen.builder.name,
            data_dir=gen.builder.config.data_dir,
        )
gen.builder._check_manual_download(dl_manager)


In [28]:
dl_manager

In [29]:
splits_generators = {sg.name: sg for sg in gen.builder._split_generators(dl_manager)}
# By default, return all splits


In [51]:
splits_generators['train'].split_info

SplitInfo(name='train', num_bytes=0, num_examples=0, shard_lengths=None, dataset_name=None)

In [54]:
res = gen.builder._get_examples_iterable_for_split(splits_generators['train'])

In [56]:
res.n_shards

5

In [35]:
from datasets.utils.py_utils import map_nested

In [37]:
# Create a dataset for each of the given splits
datasets = map_nested(
    gen.builder._as_streaming_dataset_single,
    splits_generators['train'],
    map_tuple=True,
)


In [47]:
ex_iterable = gen.builder._get_examples_iterable_for_split(splits_generators['train'])

In [62]:
list_paths

['/home/jovyan/romashka/data/val_buckets/processed_chunk_000.pkl',
 '/home/jovyan/romashka/data/val_buckets/processed_chunk_001.pkl',
 '/home/jovyan/romashka/data/val_buckets/processed_chunk_002.pkl',
 '/home/jovyan/romashka/data/val_buckets/processed_chunk_003.pkl',
 '/home/jovyan/romashka/data/val_buckets/processed_chunk_004.pkl']

In [72]:
import pickle 

for elem in list_paths:
    with open(elem, 'rb') as f:
        data = pickle.load(f)

    print(f'Reading {elem}')
    for ps in data['padded_sequences']:
        print(ps.shape)

Reading /home/jovyan/romashka/data/val_buckets/processed_chunk_000.pkl
(1021, 18, 13)
(954, 18, 20)
(783, 18, 28)
(607, 18, 35)
(667, 18, 43)
(551, 18, 50)
(580, 18, 58)
(468, 18, 65)
(491, 18, 72)
(496, 18, 80)
(404, 18, 87)
(472, 18, 95)
(375, 18, 102)
(430, 18, 110)
(354, 18, 117)
(361, 18, 125)
(310, 18, 132)
(332, 18, 139)
(370, 18, 147)
(246, 18, 154)
(347, 18, 162)
(276, 18, 169)
(309, 18, 177)
(263, 18, 184)
(297, 18, 192)
(233, 18, 199)
(259, 18, 206)
(264, 18, 214)
(214, 18, 221)
(255, 18, 229)
(206, 18, 236)
(245, 18, 244)
(212, 18, 251)
(219, 18, 258)
(222, 18, 266)
(170, 18, 273)
(200, 18, 281)
(166, 18, 288)
(195, 18, 296)
(167, 18, 303)
(188, 18, 311)
(143, 18, 318)
(154, 18, 325)
(160, 18, 333)
(151, 18, 340)
(152, 18, 348)
(130, 18, 355)
(163, 18, 363)
(136, 18, 370)
(148, 18, 378)
(130, 18, 385)
(114, 18, 392)
(138, 18, 400)
(122, 18, 407)
(133, 18, 415)
(107, 18, 422)
(118, 18, 430)
(115, 18, 437)
(96, 18, 444)
(114, 18, 452)
(87, 18, 459)
(102, 18, 467)
(108, 18, 47

In [70]:
data['padded_sequences'][0].shape

(1021, 18, 13)